In [1]:
import sqlite3
import re

import pandas
import spacy
from sklearn.metrics.pairwise import cosine_similarity
from numpy import zeros

nlp = spacy.load("en_core_web_lg")

pandas.set_option("display.max_rows", 200)

In [2]:
connection = sqlite3.connect("b2b_google_analytics.db")

In [3]:
# Joining with content_preview returns 0 results.
relevant_tables = ["keyword_search", "browse_navigation", "content_retrievals"]

df = pandas.DataFrame()
for table in relevant_tables:
    this_df = pandas.read_sql(("SELECT " + ("k.query" if table == "keyword_search" else "k.event_label") +
                               " AS user, s.event_label AS suggestion " 
                               "FROM {} k INNER JOIN suggested_search s ON "
                               "s.user_id = k.user_id AND s.event_time = k.event_time".format(table)), connection)
    this_df["table"] = table
    df = df.append(this_df, ignore_index=True)

df["ignore"] = df["user"].apply(lambda x : ("persons.code:" in x or "subjects.code:" in x
                                            or "places.code:" in x))
df = df[df["ignore"] == False]
del df["ignore"]

df

,user,suggestion,table
0,Vermont Daily Life,Vermont Daily Life,keyword_search
1,"\""Donald Trump\""",Joe Biden Donald Trump,keyword_search
2,Donald Trump portrait,Donald Trump,keyword_search
4,kelly loeffler,Republican Kelly Loeffler advances to runoff e...,keyword_search
5,real estate,Home and garden,keyword_search
6,Election 2020 Looking At America,Election 2020 Looking At America,keyword_search
8,usa,Election 2020,keyword_search
9,usa,Election 2020,keyword_search
10,missisipi,VoteCast Mississippi Race,keyword_search
11,PENNSYLVANIA ballot,PENNSYLVANIA VOTING,keyword_search


In [4]:
def get_vector(x):
    vector = zeros(300)
    for token in nlp(re.sub(r'[\'",\?\.\*\(\)\\]', "", x, re.U)):
        #print(token.text)
        if token.has_vector:
            vector = vector + token.vector
    return vector

In [5]:
def compare(u, s):
    # for addressing OOV issues, data's too old
    if u.lower() == "cawthorn" and "cawthorn" in s.lower():
        return 0.75
    if u == "aoc" and s == "Alexandria Ocasio-Cortez (politician)":
        return 1.0
    return cosine_similarity([get_vector(u)], [get_vector(s)])[0][0]

In [6]:
df["CS"] = df.apply(lambda x : compare(x["user"], x["suggestion"]), axis=1)

In [7]:
df["CS"].describe()

count    112.000000
mean       0.643127
std        0.337530
min       -0.136085
25%        0.369081
50%        0.726827
75%        1.000000
max        1.000000
Name: CS, dtype: float64

In [8]:
len(df.query('user == suggestion | CS == 1.0')) / len(df)

0.17857142857142858

In [9]:
df.query('CS > 0 & CS < 1').groupby(["table"]).agg({"CS" : "mean"})

,CS
table,
browse_navigation,0.316320
content_retrievals,0.657223
keyword_search,0.651637


In [10]:
df[df["table"] == "browse_navigation"].sort_values(by="CS")

,user,suggestion,table,CS
97,"Visuals (Photo, Video, Graphic, Audio)",EEUU ELECCIONES,browse_navigation,-0.136085
102,Top News,Sarah McBride,browse_navigation,0.081042
103,(not set),Adam Schiff (politician),browse_navigation,0.106812
98,"Visuals (Photo, Video, Graphic, Audio)",Election 2020 Melania Trump,browse_navigation,0.124434
96,"Visuals (Photo, Video, Graphic, Audio)",Election 2020 Philadelphia,browse_navigation,0.147255
106,All Newest Photos,VIENNA TERROR ATTACK,browse_navigation,0.226766
101,(not set),Cannabis (plants),browse_navigation,0.274294
104,California State News,Adam Schiff (politician),browse_navigation,0.307815
105,(not set),Entertainment,browse_navigation,0.334776
100,VoteCast Coverage,USA Presidential Election,browse_navigation,0.346832


In [11]:
df[df["table"] == "content_retrievals"].sort_values(by="CS")

,user,suggestion,table,CS
109,"53282a7da4f3a1903358b4b16d8a17a6, 41fe2b185f9d...",Austria Vienna Attack,content_retrievals,0.000000
114,US--Election 2020-Senate-Rocky Mountains,Cory Gardner (politician),content_retrievals,0.181127
121,Election 2020 Protest Washington DC,White House,content_retrievals,0.342487
107,US--APNewsAlert,Republican Mitch McConnell wins reelection to ...,content_retrievals,0.376497
123,Election 2020 Trump,Donald Trump,content_retrievals,0.486370
120,Election 2020 Trump,Donald Trump,content_retrievals,0.486370
119,"With quarterback Garbers leading the way, Cal ...",California Golden Bears football,content_retrievals,0.513090
108,US--Election 2020-Senate-The Latest,Republican Mitch McConnell wins reelection to ...,content_retrievals,0.535699
115,Cruise Ships-No Sail,Cruise industry throws in the towel on 2020 lo...,content_retrievals,0.580488
129,Nicaragua Hurricane Eta,Powerful Hurricane Eta threatens flooding in C...,content_retrievals,0.588003


In [12]:
df[df["table"] == "keyword_search"].sort_values(by="CS")

,user,suggestion,table,CS
43,AUSTRAIA,Austria Vienna Attack,keyword_search,0.000000
44,AUSTRAIA,Austria Vienna Attack,keyword_search,0.000000
39,laturner,Missouri (State),keyword_search,0.000000
10,missisipi,VoteCast Mississippi Race,keyword_search,0.000000
25,Klopp,Presidential election in the USA,keyword_search,0.000000
81,adriano espaillat,Democrat Adriano Espaillat wins reelection to ...,keyword_search,0.043228
85,nypd,NY Bracing for Election Unrest Stores are Boar...,keyword_search,0.129856
8,usa,Election 2020,keyword_search,0.139934
9,usa,Election 2020,keyword_search,0.139934
32,new jersey,Cannabis (plants),keyword_search,0.177192
